In [ ]:
import getpass
import os

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

page_content='uuid: dca9674273a7576fe3efa7e28e5c7813eb40265f373fa73da5d66d98f83cbdfa
poi_name: Sultan Restaurant
lat: 29.540509
lng: 30.979346
reviews_no: 1.0
price_range: 
category: Restaurant' metadata={'source': 'F:/AI_APPS/Tourist_Assistant/Notebooks/cairo_restaurants_cleaned.csv', 'row': 0}


## Creating Index vector database with FAISS

In [ ]:

#Loading CSV files with LangChain
from langchain.document_loaders import CSVLoader

csv_path = r"F:/AI_APPS/Tourist_Assistant/Notebooks/cairo_restaurants_cleaned.csv"

# Load CSV rows as documents
loader = CSVLoader(file_path=csv_path,csv_args={"delimiter": ",", "quotechar": '"'},encoding="utf-8")
documents = loader.load()

# Optional: print first document
print(documents[0])

# Step 2: Split into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # Max size of each chunk
    chunk_overlap=50     # Overlap between chunks to preserve context
)
chunks = text_splitter.split_documents(documents)

# Optional: See the first chunk
print(chunks[0].page_content)
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
# Step 2: Create embeddings
embeddings = OpenAIEmbeddings()  # You must have OPENAI_API_KEY set in env

# Step 3: Store chunks in FAISS vector store
vectorstore = FAISS.from_documents(chunks, embeddings)

# Optional: Save vectorstore locally
vectorstore.save_local("faiss_index")


In [6]:
# Load PDF files
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Load embeddings
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# Load PDF
loader = PyPDFLoader(r"F:\AI_APPS\Tourist_Assistant\Data\Rules.pdf")
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

## Use Hugging Face Embeddings (e.g., all-MiniLM-L6-v2)
#embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
#embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Create vectorstore with FAISS
vectorstore = FAISS.from_documents(split_docs, embeddings)

# Save the vectorstore locally
vectorstore.save_local("Rules_final_vectorstore")


In [7]:
# Load PDF files
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Load embeddings
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# Load PDF
loader = PyPDFLoader(r"F:\AI_APPS\Tourist_Assistant\Data\egypt-brochure.pdf")
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

## Use Hugging Face Embeddings (e.g., all-MiniLM-L6-v2)
#embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
#embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)

# Create vectorstore with FAISS
vectorstore = FAISS.from_documents(split_docs, embeddings)

# Save the vectorstore locally
vectorstore.save_local("brochure_final_vectorstore")


## Loading Faiss index and testing model

In [8]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
embeddings = OpenAIEmbeddings() 
# Step 1: Load your saved vector store
vectorstore = FAISS.load_local("faiss_index", OpenAIEmbeddings(),allow_dangerous_deserialization=True)

# Step 2: Initialize retriever
retriever = vectorstore.as_retriever()

# Step 3: Build QA chain using retriever + LLM
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),  # You can also use OpenAI(model="gpt-3.5-turbo")
    retriever=retriever,
    return_source_documents=True
)

# Step 4: Ask a question
query = "What is the restaurant which have good meet?"
response = qa_chain(query)

# Print answer
print("Answer:", response['result'])


C:\Users\hanye\AppData\Local\Temp\ipykernel_20568\681860109.py:14: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(),  # You can also use OpenAI(model="gpt-3.5-turbo")
C:\Users\hanye\AppData\Local\Temp\ipykernel_20568\681860109.py:21: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = qa_chain(query)


Answer: Based on the context provided, "Butcher and restaurant Well done" might be a good option for a restaurant that serves good meat.


In [ ]:
# Import necessary libraries from LangChain
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
# Ensure you have the OpenAI API key set in your environment
 
embeddings = OpenAIEmbeddings() 

# Load the webpage
url = "https://www.atlys.com/blog/rules-to-follow-in-egypt"  # Replace with your URL
loader = WebBaseLoader(url)
documents = loader.load()

# Generate embeddings for the documents using OpenAI's embeddings model
embeddings = OpenAIEmbeddings()

# Create a FAISS index to store the embeddings
vector_store = FAISS.from_documents(documents, embeddings)

# Save the FAISS index to disk (optional)
vector_store.save_local("faiss_index_rules_egypt")

# Set up a retriever to retrieve relevant documents from the vector store
retriever = vector_store.as_retriever()

# Initialize the OpenAI model (or any other LLM you prefer)
llm = OpenAI(temperature=0)

# Set up the RetrievalQA chain with the retriever and LLM
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

# Define a query to retrieve the answer from the documents
query = "Entry requirements for Egypt?"  # Example query

# Get the answer from the model
answer = qa_chain.run(query)

# Print the answer
print(answer)


C:\Users\hanye\AppData\Local\Temp\ipykernel_22760\3689664857.py:24: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()
C:\Users\hanye\AppData\Local\Temp\ipykernel_22760\3689664857.py:45: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)


In [ ]:
documents

[Document(metadata={'source': 'https://www.atlys.com/blog/rules-to-follow-in-egypt', 'title': 'Attention Required! | Cloudflare', 'language': 'en-US'}, page_content='\n\n\n\n  \n\nAttention Required! | Cloudflare\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPlease enable cookies.\n\n\nSorry, you have been blocked\nYou are unable to access atlys.com\n\n\n\n\n\n\n\n\n\n\n\nWhy have I been blocked?\nThis website is using a security service to protect itself from online attacks. The action you just performed triggered the security solution. There are several actions that could trigger this block including submitting a certain word or phrase, a SQL command or malformed data.\n\n\nWhat can I do to resolve this?\nYou can email the site owner to let them know you were blocked. Please include what you were doing when this page came up and the Cloudflare Ray ID found at the bottom of this page.\n\n\n\n\n\nCloudflare Ray ID: 93d8323d2b9ec986\n•\n\n      Your IP:\n      Click to reveal\n37.224.52.18\n•\n\nPerform

In [10]:
!cd

f:\AI_APPS\Tourist_Assistant\Notebooks


## Testing the model with Routing for multi document retrieval and data storage


In [ ]:
# Routing

In [9]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

#embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
#embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
embeddings = OpenAIEmbeddings() 

# Load vectorstores
pdf_vectorstore = FAISS.load_local("Rules_final_vectorstore", embeddings,allow_dangerous_deserialization=True)  # assuming you saved it similarly
csv_vectorstore = FAISS.load_local("brochure_final_vectorstore", embeddings,allow_dangerous_deserialization=True)  # assuming you saved it similarly


In [11]:
from langchain_community.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

# Import necessary libraries from LangChain
from langchain.document_loaders import WebBaseLoader
from langchain.embeddings import OpenAIEmbeddings

from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Initialize the OpenAI model (or any other LLM you prefer)
llm = OpenAI(temperature=0)



# Updated prompt templates with {context}
Rules_template = """You are a knowledgeable lawyer specializing in tourism laws in Egypt.
Use the following context to help answer the question:
{context}

Question:
{query}"""

restaurant_template = """You are a highly experienced tour guide specializing in restaurants.
Use the following context to help answer the question:
{context}

Question:
{query}"""

template_map = {
    "rules": PromptTemplate.from_template(Rules_template),
    "restaurant": PromptTemplate.from_template(restaurant_template),
}

retrievers = {
    "rules": pdf_vectorstore.as_retriever(search_kwargs={"k": 4}),
    "restaurant": csv_vectorstore.as_retriever(search_kwargs={"k": 4}),
}

prompt_templates = [Rules_template, restaurant_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Routing
def prompt_router(input):
    query = input.get("query", "")
    if not query or not isinstance(query, str):
        raise ValueError("Query must be a valid string")

    query_embedding = embeddings.embed_query(query)
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    source_key = "rules" if most_similar == Rules_template else "restaurant"

    print(f"🔀 Routed to: {source_key}")
    retriever = retrievers[source_key]
    context = "\n".join(doc.page_content for doc in retriever.get_relevant_documents(query))

    return {
        "query": query,
        "context": context,
        "prompt": template_map[source_key]
    }

# Final chain
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | RunnableLambda(lambda input: input["prompt"].format(query=input["query"], context=input["context"]))
    | llm
    | StrOutputParser()
)


C:\Users\hanye\AppData\Local\Temp\ipykernel_20568\923730309.py:20: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0)


In [12]:
# Test the chain
response = chain.invoke("What are the tourist guidelines in Egypt regarding photography?")
print(response)

🔀 Routed to: rules


C:\Users\hanye\AppData\Local\Temp\ipykernel_20568\923730309.py:65: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  context = "\n".join(doc.page_content for doc in retriever.get_relevant_documents(query))




Answer:
Photography is allowed in many sites in Egypt, but flash photography is prohibited in museums and tombs as it can damage artifacts. It is also important to ask for consent before taking pictures of locals and to be mindful of restricted areas such as military installations, airports, and some museums. In mosques, it is important to remove shoes before entering, dress conservatively, and observe silence or speak in a low voice. It is also illegal for foreign visitors to engage in any form of political activity, including photographing protests. Ride-hailing apps like Uber and Careem are popular and safe for navigating cities, but traditional taxis may try to overcharge tourists.


In [3]:
from langchain_community.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import OpenAIEmbeddings # Assuming you have this configured
from langchain_openai import ChatOpenAI # Assuming you have this configured

# Mocking these for demonstration purposes if you don't have them set up
# In your actual code, these would be initialized correctly
class MockVectorstore:
    def as_retriever(self, search_kwargs):
        class MockRetriever:
            def get_relevant_documents(self, query):
                # Simulate document retrieval
                if "tourist guidelines" in query.lower():
                    return [
                        type('obj', (object,), {'page_content': "Photography is generally allowed for personal use in public spaces, but not in certain religious sites or military zones without permission."})
                    ]
                elif "restaurants" in query.lower():
                    return [
                        type('obj', (object,), {'page_content': "Cairo offers a wide range of restaurants, from traditional Egyptian cuisine to international options. Tipping is customary."})
                    ]
                return []
        return MockRetriever()

class MockEmbeddings:
    def embed_documents(self, texts):
        # Simulate embedding for prompt templates
        return [[0.1, 0.2] if "rules" in text else [0.3, 0.4] for text in texts]

    def embed_query(self, text):
        # Simulate embedding for queries
        if "photography" in text.lower() or "guidelines" in text.lower():
            return [0.15, 0.25] # Closer to rules
        return [0.35, 0.45] # Closer to restaurants

# Initialize mock components
embeddings = MockEmbeddings()
pdf_vectorstore = MockVectorstore()
csv_vectorstore = MockVectorstore()
llm = ChatOpenAI(model="gpt-4o", temperature=0) # Replace with your actual LLM instance

# Updated prompt templates with {context}
Rules_template = """You are a knowledgeable lawyer specializing in tourism laws in Egypt.
Use the following context to help answer the question:
{context}

Question:
{query}"""

restaurant_template = """You are a highly experienced tour guide specializing in restaurants.
Use the following context to help answer the question:
{context}

Question:
{query}"""

template_map = {
    "rules": PromptTemplate.from_template(Rules_template),
    "restaurant": PromptTemplate.from_template(restaurant_template),
}

retrievers = {
    "rules": pdf_vectorstore.as_retriever(search_kwargs={"k": 1}),
    "restaurant": csv_vectorstore.as_retriever(search_kwargs={"k": 2}),
}

prompt_templates = [Rules_template, restaurant_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

# Routing
def prompt_router(input_dict): # Renamed 'input' to 'input_dict' for clarity
    query = input_dict.get("query", "")
    if not query or not isinstance(query, str):
        raise ValueError("Query must be a valid string")

    query_embedding = embeddings.embed_query(query)
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar_template_content = prompt_templates[similarity.argmax()] # Get the actual template string
    source_key = "rules" if most_similar_template_content == Rules_template else "restaurant"

    print(f"🔀 Routed to: {source_key}")
    retriever = retrievers[source_key]
    context = "\n".join(doc.page_content for doc in retriever.get_relevant_documents(query))

    return {
        "query": query,
        "context": context,
        "prompt": template_map[source_key]
    }

# Final chain
chain = (
    # This ensures the input to prompt_router is always {"query": <your_string_query>}
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    # The next RunnableLambda now expects a dictionary with 'prompt', 'query', and 'context'
    | RunnableLambda(lambda x: x["prompt"].format(query=x["query"], context=x["context"]))
    | llm
    | StrOutputParser()
)

# Test the chain
response = chain.invoke(
    "What are the tourist guidelines in Egypt regarding photography?"
)

print(response)

# Test with a restaurant query
response_restaurant = chain.invoke(
    "Suggest a good traditional Egyptian restaurant in Cairo."
)
print(response_restaurant)

🔀 Routed to: rules
In Egypt, tourists are generally allowed to take photographs for personal use in public spaces. However, there are important exceptions to be aware of. Photography is typically restricted in certain religious sites, such as mosques and churches, where permission may be required. Additionally, taking photographs in military zones or of military personnel and installations is prohibited without explicit permission. It's always advisable for tourists to look for signage indicating photography restrictions and to ask for permission when in doubt, especially in sensitive areas. Respecting these guidelines helps ensure a respectful and lawful visit.
🔀 Routed to: rules
While I specialize in tourism laws, I can certainly suggest a popular traditional Egyptian restaurant in Cairo. One highly recommended option is Abou El Sid. It is well-known for its authentic Egyptian cuisine and offers a wide variety of traditional dishes in a charming, vintage setting. Enjoy your meal!


In [ ]:
# Test with a restaurant query
response_restaurant = chain.invoke(
    "Suggest a good traditional Egyptian restaurant in Cairo."
)
print(response_restaurant)